In [11]:
from pyspark.ml import PipelineModel
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, regexp_replace
from pyspark.ml.feature import StringIndexer
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [12]:
from bs4 import BeautifulSoup
import requests
import re

In [7]:
spark = SparkSession.builder \
    .appName("SentimentAnalysis") \
    .getOrCreate()

In [9]:
sc = spark.sparkContext

In [10]:
print(f"Master: {sc.master}")  # In ra thông tin master
print(f"Application ID: {sc.applicationId}")  # ID của ứng dụng Spark

Master: local[*]
Application ID: local-1740635688579


In [5]:
loaded_model = PipelineModel.load("../models/sentiment_model_spark")

In [4]:
# URL bài báo
url = "https://vnexpress.net/cong-ty-chung-khoan-thuoc-nhom-flc-bi-dinh-chi-mua-chung-khoan-4851742.html"

# Gửi yêu cầu HTTP
headers = {"User-Agent": "Mozilla/5.0"}
response = requests.get(url, headers=headers)
# Kiểm tra phản hồi
if response.status_code == 200:
    # Phân tích HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Tìm các thẻ <p> có class "Normal"
    paragraphs = soup.find_all("p", class_="Normal")

    # Lấy nội dung văn bản
    content = "\n".join([p.get_text() for p in paragraphs])
else:
    print("Không thể lấy nội dung bài báo.")


In [5]:
content = content.lower()
content = re.sub(r'[^\w\s]', ' ', content)
print(content)

sở giao dịch chứng khoán việt nam  vnx  vừa đình chỉ một phần hoạt động giao dịch với công ty cổ phần chứng khoán bos  art   các hoạt động bao gồm mua chứng khoán trên thị trường niêm yết và thị trường đăng ký giao dịch tại sở giao dịch chứng khoán tp hcm  hose  và sở giao dịch chứng khoán hà nội  hnx  
nguyên nhân là doanh nghiệp này bị ủy ban chứng khoán nhà nước  ssc  đặt vào tình trạng kiểm soát  trước đó vào tháng 10 2024  bos bị kiểm soát do không đáp ứng chỉ tiêu an toàn tài chính  theo quyết định mới  công ty sẽ bị đình chỉ mua chứng khoán từ ngày 17 2 đến khi thoát khỏi tình trạng trên 
chứng khoán bos tiền thân là chứng khoán artex  thành lập năm 2008  doanh nghiệp này được biết đến là công ty liên quan đến nhiều lãnh đạo flc 
thời gian qua  bos nhận nhiều án phạt  tháng 2 2024  công ty bị phạt hơn 1 tỷ đồng do những vi phạm về giấu thông tin giao dịch với nhóm cá nhân người nhà  flc  khoảng 3 tháng sau  họ đã bị ssc thu hồi chứng nhận đủ điều kiện kinh doanh chứng khoán phái

In [12]:
data = spark.createDataFrame([(content,)], ["clean_content"])

In [8]:
predictions = loaded_model.transform(data)

25/02/26 07:00:13 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.


In [9]:
predictions.select("prediction").show(truncate=False)

+----------+
|prediction|
+----------+
|0.0       |
+----------+



In [14]:
import os
import pymongo
from pymongo import MongoClient
mongo_uri = os.environ.get('MONGO_URI')
client = MongoClient(mongo_uri)
db = client.get_database()
test_collection = db['clean_news']

In [15]:
#predict all data and save in new collection (pred_news) 
if 'pred_news' not in db.list_collection_names():
    db.create_collection('pred_news')

In [16]:
data = list(test_collection.find())
print(f"{len(data)}")

300


In [17]:
def map_prediction_to_sentiment(pred_value):
    if pred_value == 0.0:
        return "negative"
    elif pred_value == 1.0:
        return "positive"
    elif pred_value == 2.0:
        return "neutral"
    else:
        return "unknown"  # In case there are unexpected values

In [20]:
pred_collection = db['pred_news']
for doc in data:
    # Extract the content for prediction
    content = doc.get('clean_content', '')
    
    # Create DataFrame with the content
    df = spark.createDataFrame([(content,)], ["clean_content"])
    
    # Make prediction
    prediction = loaded_model.transform(df)
    
    # Extract the prediction value
    pred_value = prediction.select("prediction").collect()[0][0]
    
    # Map numeric prediction to sentiment label
    pred_sentiment = map_prediction_to_sentiment(pred_value)
    
    # Create new document for pred_news collection
    new_doc = {
        "title": doc.get("title", ""),
        "url": doc.get("url", ""),
        "pred_sentiment": pred_sentiment,
        "sentiment": doc.get("sentiment", "")  # Original sentiment label if available
    }
    
    # Insert into pred_news collection
    pred_collection.insert_one(new_doc)

25/02/26 07:36:58 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:36:59 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:37:00 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:37:00 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:37:01 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:37:01 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel for this column.
25/02/26 07:37:01 WARN StringIndexerModel: Input column sentiment does not exist during transformation. Skip StringIndexerModel fo

In [24]:
print(f"Processed and saved {pred_collection.count_documents({})} documents to pred_news collection")
print(f"Positive predictions: {pred_collection.count_documents({'pred_sentiment': 'positive'})}")
print(f"Neutral predictions: {pred_collection.count_documents({'pred_sentiment': 'neutral'})}")
print(f"Negative predictions: {pred_collection.count_documents({'pred_sentiment': 'negative'})}")

print(f"Positive: {test_collection.count_documents({'sentiment': 'positive'})}")
print(f"Neutral: {test_collection.count_documents({'sentiment': 'neutral'})}")
print(f"Negative: {test_collection.count_documents({'sentiment': 'negative'})}")

Processed and saved 300 documents to pred_news collection
Positive predictions: 111
Neutral predictions: 76
Negative predictions: 113
Positive: 109
Neutral: 77
Negative: 114
